#**VonHeijne algorithm**

#### Importing libraries and datasets

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import precision_recall_curve
import math

In [ ]:
positive = pd.read_csv("/content/positive_set.tsv", sep="\t")
negative = pd.read_csv("/content/negative_set.tsv", sep="\t")

positive["type"] = "Positive"
negative["type"] = "Negative"

print(positive["protein_id"].is_unique) #Checking if protein_id can be used as label
print(negative["protein_id"].is_unique)

positive = positive.set_index("protein_id")
negative = negative.set_index("protein_id")
positive = positive.rename(columns={"class": "dataset_class"})
negative = negative.rename(columns={"class": "dataset_class"})

tot = pd.concat([positive, negative])
new_tot = tot.copy()
new_tot = new_tot[["dataset_class",'cv_subset','type']]

new_tot["sequence"] = pd.Series("", index=new_tot.index)
new_tot["motif"] = pd.Series("", index=new_tot.index)

new_tot.head()

True
True


,dataset_class,cv_subset,type,sequence,motif
protein_id,,,,,
Q99MA2,Training,1,Positive,,
P17948,Training,1,Positive,,
P41271,Training,1,Positive,,
Q8I948,Training,1,Positive,,
Q92154,Training,1,Positive,,


In [ ]:
# add sequence & motif to the dataframe
positive_seq_fasta = "/content/cluster-results_positive_rep_seq.fasta"
negative_seq_fasta = "/content/cluster-results_negative_rep_seq.fasta"
positive_motif = "/content/training_motif_alignment.fasta"

with open(positive_seq_fasta,"r") as file:
  header = True
  for line in file:
    if header == True:
      ID = str(line.split()[0][1:])
      header = False
      continue
    if header == False:
      if ID in new_tot.index: # Check if ID exists in the dataframe index
        new_tot.loc[ID, "sequence"] = line.strip()
      header = True
file.close()

with open(negative_seq_fasta,"r") as file:
  header = True
  for line in file:
    if header == True:
      ID = str(line.split()[0][1:])
      header = False
      continue
    if header == False:
      if ID in new_tot.index: # Check if ID exists in the dataframe index
        new_tot.loc[ID, "sequence"] = line.strip()
      header = True
file.close()

with open(positive_motif,"r") as file:
  header = True
  for line in file:
    if header == True:
      ID = str(line.split()[0][1:])
      header = False
      continue
    if header == False:
      if ID in new_tot.index: # Check if ID exists in the dataframe index
        new_tot.loc[ID, "motif"] = line.strip()
      header = True
file.close()

original = new_tot.copy()
#test = new_tot.copy()
original = original.query('dataset_class == "Training"')
#test = test.query('dataset_class == "Test"')

new_tot.head()


,dataset_class,cv_subset,type,sequence,motif
protein_id,,,,,
Q99MA2,Training,1,Positive,MAQAYWQCYPWLVLLCACAWSYPGPESLGREDVRDCSTNPPRLPVT...,PWLVLLCACAWSYPG
P17948,Training,1,Positive,MVSYWDTGVLLCALLSCLLLTGSSSGSKLKDPELSLKGTQHIMQAG...,LLSCLLLTGSSSGSK
P41271,Training,1,Positive,MMLRVLVGAVLPAMLLAAPPPINKLALFPDKSAWCEAKNITQIVGH...,RVLVGAVLPAMLLAA
Q8I948,Training,1,Positive,MAFRMKLVVCIVLLSTLAVMSSADVYKGGGGGRYGGGRYGGGGGYG...,IVLLSTLAVMSSADV
Q92154,Training,1,Positive,MELLVLTVLLMGTGCISAPWAAWMPPKMAALSGTCVQLPCRFDYPE...,VLTVLLMGTGCISAP


In [ ]:
# split validation - training subsets -> prende df +- training
def cv_subsets(df):
  subsets = set()
  for _, row in df.iterrows():
    index = row["cv_subset"]
    subsets.add(index)
  combinations={}
  for index in subsets:
    validation = index
    training = list(subsets - {index})
    combinations[validation] = training
  return combinations


In [ ]:
def df_subset(df, combinations, index):
  l = combinations[index]
  training = l[:-1]
  test = l[-1]
  for id,row in df.iterrows():
    if row["cv_subset"] in training:
      df.loc[id, "dataset_class"] = "Training"
    elif row["cv_subset"] == test:
      df.loc[id, "dataset_class"] = "Testing"
    else:
      df.loc[id, "dataset_class"] = "Validation"
  return

original.head()

,dataset_class,cv_subset,type,sequence,motif
protein_id,,,,,
Q99MA2,Training,1,Positive,MAQAYWQCYPWLVLLCACAWSYPGPESLGREDVRDCSTNPPRLPVT...,PWLVLLCACAWSYPG
P17948,Training,1,Positive,MVSYWDTGVLLCALLSCLLLTGSSSGSKLKDPELSLKGTQHIMQAG...,LLSCLLLTGSSSGSK
P41271,Training,1,Positive,MMLRVLVGAVLPAMLLAAPPPINKLALFPDKSAWCEAKNITQIVGH...,RVLVGAVLPAMLLAA
Q8I948,Training,1,Positive,MAFRMKLVVCIVLLSTLAVMSSADVYKGGGGGRYGGGRYGGGGGYG...,IVLLSTLAVMSSADV
Q92154,Training,1,Positive,MELLVLTVLLMGTGCISAPWAAWMPPKMAALSGTCVQLPCRFDYPE...,VLTVLLMGTGCISAP


In [ ]:
# prende i fasta dei motif dei training
def matrix_generation(df):
  df = df.query('type == "Positive"')
  df_row_count = df.shape[0] # N number of rows
  shape_motif = (-13,2) #motif indexes
  #Matrix init with pseudocount
  matrix = {shape_motif[0]:{"A": 1, "Q": 1, "L": 1, "S": 1, "R": 1, "E": 1, "K": 1, "T": 1, "N": 1, "G": 1, "M": 1, "W": 1, "D": 1, "H": 1, "F": 1, "Y": 1, "C": 1, "I": 1, "P": 1, "V": 1}}
  for i in range(shape_motif[0]+1, shape_motif[1]):
    matrix[i] = {"A": 1, "Q": 1, "L": 1, "S": 1, "R": 1, "E": 1, "K": 1, "T": 1, "N": 1, "G": 1, "M": 1, "W": 1, "D": 1, "H": 1, "F": 1, "Y": 1, "C": 1, "I": 1, "P": 1, "V": 1}

  #Counting aa occurences
  for _, row in df.iterrows():
    motif = str(row["motif"])
    for index,aa in enumerate(motif):
      if aa == "X" or aa == "n" or aa == "a" or aa == "U":
        continue
      matrix[index-13][aa] += 1

  #Dividing for N + 20
  for key, value in matrix.items():
    for index, aa in matrix[key].items():
      matrix[key][index] = (aa/(df_row_count+20))

  #Transformation into a pd df
  matrix_df = pd.DataFrame.from_dict(matrix, orient='index')
  reference = {"A": 0.0825, "Q": 0.0393, "L": 0.0964, "S": 0.0665, "R": 0.0552, "E": 0.0671, "K": 0.0579, "T": 0.0536, "N": 0.0406, "G": 0.0707, "M": 0.0241, "W": 0.0110, "D": 0.0546, "H": 0.0227, "F": 0.0386, "Y": 0.0292, "C": 0.0138, "I": 0.0590, "P": 0.0474, "V": 0.0685}

  #Dividing for reference
  for _,row in matrix_df.iterrows():
    for aa, value in reference.items():
      row[aa] = row[aa]/value

  #Apply log
  matrix_df = matrix_df.apply(np.log)
  return matrix_df


In [ ]:
# predict scores from the matrix per entry
def score_prediction(matrix, entry):
  score = 0
  best_score = -100000000000000000000000000000000000000
  seq = entry["sequence"]
  l = 15
  for index in range(91):
    motif = seq[index:index+l]
    position = -13
    for residue in motif:
      if residue == "X" or residue == "n" or residue == "a" or residue == "U":
        continue
      score += matrix.loc[position][residue]
      position+=1
    if score > best_score:
      best_score = score
  return best_score

In [ ]:
# iterate to find the best threshold
def threshold_estimation(df):
  y_validation = []
  entry_types = df["type"]
  for entry in entry_types:
    if entry == "Positive":
      y_validation.append(1)
    else:
      y_validation.append(0)
  y_validation_scores = []
  for id,row in df.iterrows():
     entry_score = score_prediction(w, row)
     y_validation_scores.append(entry_score)
  precision, recall, thresholds = precision_recall_curve(y_validation, y_validation_scores)
  fscore = (2 * precision * recall) / (precision + recall)
  index = np.argmax(fscore)
  optimal_threshold = thresholds[index]
  return optimal_threshold

In [ ]:
# classification of the entry
def classification(score, threshold):
  if score >= threshold:
    entry_class = "Positive"
  else:
    entry_class = "Negative"
  return entry_class

In [ ]:
# performance metrics
def performance_metrics(df):
  TP = FN = FP = TN = 1
  for id,row in df.iterrows():
    real = row["type"]
    predicted = row["predicted"]
    if real == "Positive" and predicted == "Positive":
      TP += 1
    elif real == "Positive" and predicted == "Negative":
      FN += 1
    elif real == "Negative" and predicted == "Positive":
      FP += 1
    elif real == "Negative" and predicted == "Negative":
      TN += 1
  precision = TP / (TP + FP)
  recall = TP / (TP + FN)
  f1_score = 2*precision*recall / (precision + recall)
  accuracy = (TP + TN) / (TP + TN + FP + FN)
  mcc = (((TP * TN) - (FP * FN))/math.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN)))
  print("TP:", TP," FP:",FP," TN:",TN," FN:",FN)
  print("Precision:", precision)
  print("Recall:", recall)
  print("F1 score:", f1_score)
  print("Accuracy:", accuracy)
  print("MCC:",mcc)
  return [precision,recall,f1_score,accuracy,mcc]

#### Final execution

In [ ]:
combinations = cv_subsets(original) # validation : training
#combinations = {1: [2, 3, 4, 5]}
metrics_tot = []

for index in combinations: #for every cross validation subset
  print("Training on cv: ", combinations[index][:-1], " Validating on cv: ", index, " Testing on cv: ", combinations[index][-1])
  df_subset(original, combinations, index)
  training = original.copy()
  training = training.query('dataset_class == "Training"')
  w = matrix_generation(training)
  print("Matrix generated")
  print(w)

  print("Validation")
  validation = original.copy()
  validation = validation.query('dataset_class == "Validation"')
  validation["predicted"] = pd.Series("", index=new_tot.index)
  threshold = threshold_estimation(validation)
  print("Threshold: ",threshold)
  for id,row in validation.iterrows():
     entry_score = score_prediction(w, row)
     validation.loc[id, "predicted"] = classification(entry_score, threshold)

  print("Testing")
  testing = original.copy()
  testing = testing.query('dataset_class == "Testing"')
  testing["predicted"] = pd.Series("", index=new_tot.index)
  for id,row in testing.iterrows():
     entry_score = score_prediction(w, row)
     testing.loc[id, "predicted"] = classification(entry_score, threshold)

  print("Performance metrics on testing: ")
  metrics = performance_metrics(testing)
  metrics_tot.append(metrics)

  print("Finished")
  print()
  print("------------------------------------")
  print()

print("FINISHED EVERY CV")
for i in metrics_tot:
  print(i)

Training on cv:  [2, 3, 4]  Validating on cv:  1  Testing on cv:  5
Matrix generated
            A         Q         L         S         R         E         K  \
-13  0.183155 -1.965643  1.390321 -0.258028 -1.794556 -2.212920 -2.353136   
-12  0.201504 -3.064255  1.506523 -0.699861 -3.403993 -3.599215 -2.353136   
-11  0.164463 -1.272496  1.432091 -0.124497 -2.710846 -2.906067 -3.451748   
-10  0.085991 -1.454817  1.390321 -0.156245 -1.794556 -2.906067 -2.758601   
-9   0.799341 -1.272496  1.341768 -0.189035 -3.403993 -2.212920 -3.451748   
-8   0.737466 -0.867030  1.081888  0.301588 -2.305381 -1.989777 -2.758601   
-7   0.305045 -0.666360  1.290737  0.073329 -1.458083 -1.653304 -2.065454   
-6   0.715960 -2.371108  0.747993 -0.006714 -2.305381 -2.906067 -3.451748   
-5   0.660079  0.231582 -0.272657  0.643874 -0.695943 -0.603482 -1.372306   
-4   0.237222  0.026787  0.445183  0.398752 -0.631405 -0.603482 -1.053853   
-3   1.163984 -3.064255 -1.071165  0.713833 -2.305381 -2.500602 -2.0

KeyboardInterrupt: 